In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import numpy as np
import cv2
from PIL import Image

#import cc3d

from skimage.measure import regionprops, label, find_contours
from scipy.spatial.distance import cdist

import pandas as pd

In [ ]:
#値を-1から1に正規化する関数
def normalize_x(image):
    return image / 127.5 - 1


def denormalize_x(image):
    return (image + 1) * 127.5


#値を0から1正規化する関数
def normalize_y(image):
    return image / 255


#値を0から255に戻す関数
def denormalize_y(image):
    return image * 255

In [ ]:

# インプット画像を読み込む関数
def load_X_gray(folder_path):
    
    image_files = []

    #image_files = os.listdir(folder_path)
       
    for file in os.listdir(folder_path):
        base, ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(file)
        else :
            pass
        
    image_files.sort()
    print (image_files)
    
    img = cv2.imread(folder_path + os.sep + image_files[0], cv2.IMREAD_GRAYSCALE)
    
    #image_files = image_files[1:]
    images = np.zeros((len(image_files), img.shape[0], img.shape[1], 1), np.float32)
    for i, image_file in enumerate(image_files):
        image = cv2.imread(folder_path + os.sep + image_file, cv2.IMREAD_GRAYSCALE)
        print (image.shape)
        #image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
        image = image[:, :, np.newaxis]
        images[i] = normalize_x(image)
    return images, image_files


def load_Y_gray_with_gaussian_and_numeric(folder_path, thresh = None , normalize = True, g_size = None):
    image_files = []
    #image_files = os.listdir(folder_path)
    
    for file in os.listdir(folder_path):
        base, ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(int(base))
        else:
            pass
        
    image_files.sort()
    print(image_files)
    
    img = cv2.imread(folder_path + os.sep + str(image_files[0]).zfill(4) + ".png", cv2.IMREAD_GRAYSCALE)
    
    images = np.zeros(
        (len(image_files), img.shape[0], img.shape[1], 1) ,np.float32
    )
    
    for i , image_file in enumerate(image_files):
        image = cv2.imread(
            folder_path + os.sep + str(image_file).zfill(4) + ".png" ,
            cv2.IMREAD_GRAYSCALE
        )
        #print(image.shape)
        
        # ぼかし処理
        if g_size:
            image = cv2.GaussianBlur(
                image, (g_size, g_size), 0
            )
        
        
        
        if thresh:
            ret , image = cv2.threshold(image , thresh , 255 , cv2.THRESH_BINARY)
        image = image[ : , : , np.newaxis]
        if normalize:
            images[i] = normalize_y(image)
        else:
            images[i] = image
            
    print(images.shape)
    
    return images , image_files

In [ ]:
def make_mask_img(ori_img, mask_img):
    mask_img_rgb = np.zeros((mask_img.shape[0], mask_img.shape[1], 3), np.float32)
    mask_img_rgb[:,:,0] = mask_img[:,:,0]
    mask_img_rgb[:,:,1] = mask_img[:,:,0]
    masked_img = cv2.addWeighted(mask_img_rgb,0.5,cv2.cvtColor(ori_img + 0.75, cv2.COLOR_GRAY2BGR),0.6,0)
    return masked_img

In [ ]:
ori_imgs, _ = load_X_gray(r"Y:\DeepLearningData\research_010_NIH3T3\shOPA1_003\dataset\cropped_005\resize_10x10x10")

In [ ]:
seg_imgs_lamellar, _ = load_Y_gray_with_gaussian_and_numeric(r"Y:\DeepLearningData\research_010_NIH3T3\shOPA1_003\annotations\cropped_005\merged_cristae_final_002\lamellar_with_postprocessing")
seg_imgs_tubular, _ = load_Y_gray_with_gaussian_and_numeric(r"Y:\DeepLearningData\research_010_NIH3T3\shOPA1_003\annotations\cropped_005\merged_cristae_final_002\tubular_with_postprocessing")

In [ ]:
seg_imgs_tubular = np.where(
    seg_imgs_tubular > 0,
    255,
    0
)
seg_imgs_lamellar = np.where(
    seg_imgs_lamellar > 0,
    255,
    0
)

In [ ]:
#onionにみえるところ
N = 377
x = 326
y = 372

_ = np.zeros(ori_imgs.shape)
masked_ori_imgs = make_2_mask_img(ori_imgs[N] + 0.5, _[N], _[N])
masked_seg_imgs = make_2_mask_img(ori_imgs[N] + 0.5, _[N], seg_imgs_lamellar[N])
plt.figure(dpi=200)

plt.subplot(1, 2, 1)
plt.title("EM image")
plt.axis("off")
plt.imshow(masked_ori_imgs[x-100:x+100,y-100:y+100])

plt.subplot(1, 2,2)
plt.title("EM + seg")
plt.axis("off")
plt.imshow(masked_seg_imgs[x-100:x+100,y-100:y+100])


In [ ]:
out_dir = "Y:/paper_data/2021_cristae/onion-like"

os.makedirs(out_dir, exist_ok = True)

cv2.imwrite(
    f"{out_dir}/shOPA1_cropped05_onion_001_onion_ori.png",
    denormalize_y(masked_ori_imgs[x-100:x+100,y-100:y+100])
)
cv2.imwrite(
    f"{out_dir}/shOPA1_cropped05_onion_001_onion_seg.png",
    denormalize_y(masked_seg_imgs[x-100:x+100,y-100:y+100])
)

In [ ]:
#not floatingです
N = 399
x = 320
y = 365

_ = np.zeros(ori_imgs.shape)
masked_ori_imgs = make_2_mask_img(ori_imgs[N] + 0.5, _[N], _[N])
masked_seg_imgs = make_2_mask_img(ori_imgs[N] + 0.5, _[N], seg_imgs_lamellar[N])
plt.figure(dpi=200)

plt.subplot(1, 2, 1)
plt.title("EM image")
plt.axis("off")
plt.imshow(masked_ori_imgs[x-100:x+100,y-100:y+100])

plt.subplot(1, 2,2)
plt.title("EM + seg")
plt.axis("off")
plt.imshow(masked_seg_imgs[x-100:x+100,y-100:y+100])


In [ ]:
out_dir = "Y:/paper_data/2021_cristae/onion-like"

os.makedirs(out_dir, exist_ok = True)

cv2.imwrite(
    f"{out_dir}/shOPA1_cropped05_onion_001_arc_ori.png",
    denormalize_y(masked_ori_imgs[x-100:x+100,y-100:y+100])
)
cv2.imwrite(
    f"{out_dir}/shOPA1_cropped05_onion_001_arcseg.png",
    denormalize_y(masked_seg_imgs[x-100:x+100,y-100:y+100])
)